In [1]:
# Imports
#  /home/bradachi/Documentos/GitHub/NLP_teste/NLP_ENV/bin/python -m pip install 
import requests
import json
import pprint
import pandas as pd
import lxml
import time
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager 
from selenium.webdriver.firefox.service import Service 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException


In [2]:
df_características = pd.read_csv("caracteristicas.csv")
df_sem_duplicatas = df_características.drop_duplicates().reset_index(drop=True)

lista_nomes = df_sem_duplicatas[["name", "id"]].iloc[11522:17283]
lista_nomes

,name,id
11522,Computador desktop Cx Intel G5905 8gbssd240gb ...,MLB50658860
11523,Computador Core I3 10100 32gb 256gb + Monitor 24,MLB50535098
11524,Computador Intel I5 / 4gb Ram / Hd 500gb / Mon...,MLB36684518
11525,Computador Fácil Intel Core I3 4gb Hd 500gb,MLB37580287
11526,Computador Alphapc Intel Core I7 8gb Ssd 480gb,MLB35057087
...,...,...
17278,Caixa De Som Alexa Echo Dot 5 G Assistente Vir...,MLB25956240
17279,Smart Station Luminária Caixa De Som Carregame...,MLB39685916
17280,Luminária G Speaker Smart Station Caixa De Som...,MLB35491732
17281,Smart Station Luminária Caixa De Som Carregame...,MLB37202258


In [3]:
# não há perguntas nem avaliações??

# vamos usar selenium pra pegar esses dados

# instala o GeckoDriverManager com a versão certa do firefox
servico = Service(GeckoDriverManager().install())

navegador = webdriver.Firefox(service=servico)
navegador.get("https://www.mercadolivre.com.br/")

navegador.find_element('xpath', '/html/body/header/div/div[6]/nav/ul/li[2]/a').click()
navegador.find_element('xpath', '//*[@id="user_id"]').send_keys("joao.bradachi@gmail.com")
navegador.find_element('xpath', '//*[@id=":Rijh1:"]').click()

In [7]:
SCROLL_PAUSE_TIME = 0.5

def navega_ate_pagina(navegador, id, name):
    url = "https://www.mercadolivre.com.br/"+name.replace(" ", "-").replace("/", "").lower()+"/p/"+id
    navegador.get(url)

    try:
        navegador.find_element('xpath', '/html/body/main/div/div/div/span')
        raise Exception("url não funcionou, pesquisando...")
    except NoSuchElementException as e:
        pass
    except Exception as e:
        print(e)
        url = "https://www.mercadolivre.com.br/"    
        navegador.get(url)
        navegador.find_element('xpath', '//*[@id="cb1-edit"]').send_keys(name)
        navegador.find_element('xpath', '/html/body/header/div/div[2]/form/button').click()

        navegador.find_element('xpath', '/html/body/main/div/div[2]/section/ol/li[1]/div/div/div[2]/h3/a').click()

def pega_avaliacao(navegador, id, name, temp_df_avaliacoes):
    try: # tenta pegar avaliação
        
        button = navegador.find_element('xpath', '/html/body/main/div[2]/div[5]/div[4]/div[1]/div[2]/div/div/div/section/div/div[2]/div[3]/button')
        navegador.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
        
        time.sleep(SCROLL_PAUSE_TIME*2)

        button.click()

        time.sleep(SCROLL_PAUSE_TIME*2)
        
        iframe = navegador.find_element('xpath', '//*[@id="ui-pdp-iframe-reviews"]')
        
        navegador.switch_to.frame(iframe)

        # Get scroll height
        last_height = navegador.execute_script("return document.body.scrollHeight")

        while True:
            # Scroll down to bottom
            navegador.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME*2)

            # Calculate new scroll height and compare with last scroll height
            new_height = navegador.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        texto = navegador.find_element('xpath', '/html/body/main/div/section/div/div[2]/div[2]').text.split("\nÉ útil\n0\nMais opções\n")
        
        # trata texto
        alv = texto[0].split('\n')
        alv = alv[4:]
        if "Resumo de opiniões gerado por IA" in alv:
            temp_df_avaliacoes = pd.concat([pd.DataFrame({"texto": [alv[0], alv[4]], 
                                                            "nota": ['5', alv[2].split()[1]], 
                                                            "id": [id, id],
                                                            "data":["20 nov. 2025", alv[3]]}), temp_df_avaliacoes])
        else:
            alv = texto[0].split('\n')
            alv = alv[4:]
            temp_df_avaliacoes = pd.concat([pd.DataFrame({"texto": [alv[0]], 
                                                            "nota": [alv[2].split()[1]], 
                                                            "id": [id],
                                                            "data":[alv[3]]}), temp_df_avaliacoes])
        
        texto = texto[1:]
        for avaliacao in texto:
            alv = avaliacao.split('\n')

            if len(alv) == 3:
                temp_df_avaliacoes = pd.concat([pd.DataFrame({"texto": [alv[2]], 
                                                            "nota": [alv[0].split()[1]], 
                                                            "id": [id],
                                                            "data":[alv[1]]}), temp_df_avaliacoes])
            if len(alv) == 2:
                temp_df_avaliacoes = pd.concat([pd.DataFrame({"texto": [None], 
                                                            "nota": [alv[0].split()[1]], 
                                                            "id": [id],
                                                            "data":[alv[1]]}), temp_df_avaliacoes])
        
        navegador.find_element('xpath', '//*[@id=":R17h:"]').click()
        navegador.switch_to.default_content()

    except NoSuchElementException as e:
        print(e)
        print(f"Não existe menu de avaliação para o objeto {name}, {id}, testando outra forma...")

        try: # tenta pegar avaliação    TODO
            texto = navegador.find_element('xpath', '/html/body/main/div[2]/div[5]/div[4]/div[1]/div[2]/div/div/div/section/div/div[2]/div[1]/div/div').text.split('\n')
            
            to_remove = ['É útil', 'Mais opções']
            texto = [item for item in texto if item not in to_remove]
            if "Ainda não há comentários sobre este produto." in texto:
                raise Exception("Não existe comentários sobre o produto")
            for x in range(0, len(texto), 4):
                temp_df_avaliacoes = pd.concat([pd.DataFrame({"texto": [texto[x+2]], 
                                                            "nota": [texto[x].split()[1]], 
                                                            "id": [id],
                                                            "data":[texto[x+1]]}), temp_df_avaliacoes])

        except NoSuchElementException as e:
            print(e)
            print(f"Não existe avaliação para o objeto {name}, {id}")

        except Exception as e:
            print(e)
            print(f"Não existe avaliação para o objeto {name}, {id}")
    except Exception as e:
        print(e)
        print(f"Erro desconhecido no objeto {name}, {id}")
    
    return temp_df_avaliacoes

def pega_pergunta(navegador, id, name, temp_df_perguntas):
    try: # tenta pegar avaliação
        
        # q = navegador.find_element('xpath', '//*[@id="questions"]')
        button = navegador.find_element('xpath', '//*[@id="questions"]/div[2]/div/div/div/a')
        navegador.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
        
        time.sleep(SCROLL_PAUSE_TIME*2)
        
        questions = button.get_attribute("href")

        navegador.get(questions)

        time.sleep(SCROLL_PAUSE_TIME*2)

        # Get scroll height
        last_height = navegador.execute_script("return document.body.scrollHeight")

        while True:
            # Scroll down to bottom
            navegador.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME*2)

            # Calculate new scroll height and compare with last scroll height
            new_height = navegador.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        texto = navegador.find_element('xpath', '/html/body/main/div/div/div/div/div/div/div').text.split('\n')

        for x in range(0, len(texto), 3):
            temp_df_perguntas = pd.concat([pd.DataFrame({"pergunta": [texto[x]], 
                                                            "resposta": [texto[x+1]], 
                                                            "id": [id],
                                                            "data":[texto[x+2]]}), temp_df_perguntas])
    except NoSuchElementException as e:
        print(e)
        print("não encontrou o botão de mais perguntas")
        try:
            texto = navegador.find_element('xpath', '//*[@id="qadb"]').text.split('\n')
            # TODO verificar esse método
            print(texto)
            for x in range(len(texto), 3):
                temp_df_perguntas = pd.concat([pd.DataFrame({"pergunta": [texto[x]], 
                                                            "resposta": [texto[x+1]], 
                                                            "id": [id],
                                                            "data":[texto[x+1]]}), temp_df_perguntas])

        except NoSuchElementException as e:
            print(e)
            print(f"Não existe perguntas para o objeto {name}, {id}")
    return temp_df_perguntas
    

In [8]:
def busca_dados(navegador, arq_avaliacoes, arq_perguntas):
    primeiro_d = True
    for row in lista_nomes.itertuples():
        temp_df_perguntas = pd.DataFrame({"pergunta":[], "resposta":[], "id":[], "data": []})
        temp_df_avaliacoes = pd.DataFrame({"texto":[], "nota":[], "id":[], "data": []})

        navega_ate_pagina(navegador, row.id, row.name)

        temp_df_avaliacoes = pega_avaliacao(navegador, row.id, row.name, temp_df_avaliacoes)

        temp_df_perguntas = pega_pergunta(navegador, row.id, row.name, temp_df_perguntas)  

        if primeiro_d:
            temp_df_avaliacoes.to_csv(arq_avaliacoes, index=False)
            temp_df_perguntas.to_csv(arq_perguntas, index=False)
            primeiro_d=False
        else:
            temp_df_avaliacoes.to_csv(arq_avaliacoes, index=False, header=False, mode='a')
            temp_df_perguntas.to_csv(arq_perguntas, index=False, header=False, mode='a')

        # display(temp_df_avaliacoes)
        # display(temp_df_perguntas)
        

In [9]:
busca_dados(navegador, "avaliacoes3.csv", "perguntas3.csv")

Message: Unable to locate element: /html/body/main/div[2]/div[5]/div[4]/div[1]/div[2]/div/div/div/section/div/div[2]/div[3]/button; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:202:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:555:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Não existe menu de avaliação para o objeto Computador desktop Cx Intel G5905 8gbssd240gb S/Soper, MLB50658860, testando outra forma...
Message: Unable to locate element: /html/body/main/div[2]/div[5]/div[4]/div[1]/div[2]/div/div/div/section/div/div[2]/div[1]/div/div; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#nosuchelementexception
Stacktrace:
Rem

KeyboardInterrupt: 